<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%EC%86%8C%ED%94%84%ED%8A%B8%EB%A7%A5%EC%8A%A4_%ED%9A%8C%EA%B7%80_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 소프트맥스 회귀의 비용함수 구현하기

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
z = torch.FloatTensor([1,2,3])    # input

In [4]:
hypothesis = F.softmax(z, dim=0)  # softmax
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
z = torch.rand(3,5, requires_grad=True)   # 임의의 3 x 5 행렬의 크기를 가진 텐서 생성

In [6]:
hypothesis= F.softmax(z, dim=1)   # 각 행의 원소들의 합이 1이 되는 텐서로 변환
print(hypothesis)                 # 소프트맥스 함수의 출력값은 결국 예측값 ( 즉, 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지 예측한 결과 )

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [8]:
y = torch.randint(5, (3,)).long()   # 각 샘플에 대해서 임의의 레이블 생성
print(y)

tensor([2, 1, 0])


In [9]:
# 원 핫 인코딩
y_one_hot = torch.zeros_like(hypothesis)   # 모든 원소가 0의 값을 가진 3x5 텐서 생성
y_one_hot.scatter_(1, y.unsqueeze(1),1)    # y.unsqueeze(1)를 하면 y 텐서 : (3,) -> (3 x 1) 텐서
                                           # scatter의 첫번째 인자 : dim=1에 대해서 수행
                                                       세번째 인자 : 숫자 1을 넣어주므로서 두번째 인자인 y_unsqueeze(1)이 알려주는 위치에 숫자 1을 넣도록 한다.

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [10]:
print(y.unsqueeze(1))  

tensor([[2],
        [1],
        [0]])


In [11]:
print(y_one_hot)

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])


In [12]:
# 비용함수 구현
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6682, grad_fn=<MeanBackward0>)


### 파이토치로 소프트맥스의 비용함수 구현하기 ( high-level)

1.    F.softmax() + torch.log() = F.log_softmax()
2.    F.log_softmax() + F.nll_loss() = F.cross_entropy()



In [13]:
F.log_softmax(z, dim=1)    # 1

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [14]:
F.nll_loss(F.log_softmax(z,dim=1),y)  # 2       # nll :Negative Log Likelihood의 약자

tensor(1.6682, grad_fn=<NllLossBackward0>)

In [15]:
F.cross_entropy(z,y)   # 2  # F.cross_entropy는 비용함수에 소프트맥스 함수까지 포함하고 있음

tensor(1.6682, grad_fn=<NllLossBackward0>)

### 소프트맥스 회귀 구현하기

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
torch.manual_seed(1)

In [18]:
x_train = [[1, 2, 1, 1],    # 각 샘플은 4개의 특성을 가지고 있으며, 총 8개의 샘플이 존재
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]   # 총 3개의 class가 존재
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [19]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


y_train의 크기는 8x1이다. 그런데 최종 사용할 레이블은 y_train에서 원-핫 인코딩을 한 결과여야 한다.

클래스의 개수는 3개이므로, y_train에 원-핫 인코딩한 결과는 8x3 개수를 가져야 한다.

In [20]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
print(y_one_hot.shape)

torch.Size([8, 3])


y_train에서 원-핫 인코딩을 한 결과인 y_one_hot의 크기는 8x3이다. 

즉, W행렬의 크기는 4 x 3 이어야 한다.

In [21]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [24]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  
  # cost 계산
  z = x_train.matmul(W) + b
  cost = F.cross_entropy(z, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### 소프트맥스 회귀 nn.Module로 구현하기


In [25]:
model = nn.Linear(4,3)   # 모델을 선언 및 초기화 / 4개의 특성을 가지고 3개의 클래스로 분류 / input_dim =4, output_dim =3

In [26]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  prediction = model(x_train)   # H(x) 계산
  cost = F.cross_entropy(prediction, y_train)   # cost계산
  # cost로 H(x) 개선
  optimizer.zero_grad()    
  cost.backward()
  optimizer.step()
  # 20번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


### 소프트맥스 회귀 클래스로 구현하기
소프트맥스 회귀를 nn.Module을 상속받은 클래스로 구현

In [27]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4,3)  
  def forward(self, x):
    return self.linear(x)  

In [28]:
model = SoftmaxClassifierModel()